In [14]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_all = fetch_20newsgroups(data_home='.', subset='all')
newsgroups_train = fetch_20newsgroups(data_home='.', subset='train')
newsgroups_test = fetch_20newsgroups(data_home='.', subset='test')

In [19]:
print(newsgroups_all.filenames.shape)
print(newsgroups_train.filenames.shape)
print(newsgroups_test.filenames.shape)

(18846,)
(11314,)
(7532,)


In [6]:
print(dir(newsgroups_all))

['DESCR', 'data', 'description', 'filenames', 'target', 'target_names']


In [18]:
print(newsgroups_all.data[0])
print(newsgroups_all.target_names[0])

the 20 newsgroups by date dataset
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


alt.atheism
